In [4]:
%pip install pandas scikit-learn pandas xgboost caimcaim

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
from IPython.display import  clear_output
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
import math
from collections import defaultdict
from sklearn.metrics import accuracy_score
from scipy.io import loadmat
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from caimcaim import CAIMD
import time
from sklearn.metrics import log_loss

In [6]:
def entropy(X):
    unique, count = np.unique(X, return_counts=True, axis=0)
    prob = count / len(X)
    en = np.sum((-1) * prob * np.log2(prob))
    return en

In [7]:
# Joint Entropy H(x,y)
def joint_entropy(X, Y):
    XY = np.c_[X, Y]
    return entropy(XY)


In [8]:
# Joint Entropy H(x,y,z)
def joint_entropy_3(X, Y, Z):
    XYZ = np.c_[X, Y, Z]
    return entropy(XYZ)

In [9]:
# Conditional Entropy X given Y; H(x|y)
def conditional_entropy(X, Y):
    return joint_entropy(X, Y) - entropy(Y)

In [10]:
def ret_joint_entropy3(x,y):
    if(x<y):
        return joint_entropy_list3[x][y]
    else:
        return joint_entropy_list3[y][x]

In [11]:

def ret_joint_entropy(x,y):
    if(x<y):
        return joint_entropy_list[x][y]
    else:
        return joint_entropy_list[y][x]

In [12]:
#### H(x,y|z)
def conditional_join_entropy(x, y, z):
    if x == length - 1:
        return ret_joint_entropy3(y, z) - entropy_list[z];
    elif y == length - 1:
        return ret_joint_entropy3(x, z) - entropy_list[z];
    else:
        return ret_joint_entropy3(x, y) - entropy_list[z];

In [13]:
##### I(x,y)
def get_mutual_info(x, y):
    return entropy_list[x] + entropy_list[y] - ret_joint_entropy(x, y);

In [14]:
########## I(x,y|z)
def conditional_mutual_info(x, y, z):
    cxz = ret_joint_entropy(x, z)-entropy_list[z];
    cyz = ret_joint_entropy(y, z)-entropy_list[z];
    return cxz + cyz - conditional_join_entropy(x, y, z)


In [15]:
def feature_name(arr):
    name = "{";
    for i in range(len(arr)):
        if i > 0:
            name = name + ", ";

        name = name + feature_list[arr[i]]

    name = name + "}"
    return name

In [16]:
def feature_array(arr):
    name = []
    for i in range(len(arr)):
        name.append(feature_list[arr[i]])
    return name

In [17]:
def return_corelation(x, y):
    size = len(x);
    ux = x.sum() / size
    uy = y.sum() / size

    xmux = x - ux
    ymuy = y - uy

    xmuxymuy = xmux * ymuy;

    cov = xmuxymuy.sum() / (size - 1)

    var_x = xmux * xmux
    var_x = var_x.sum() / (size - 1)

    var_y = ymuy * ymuy
    var_y = var_y.sum() / (size - 1)

    sd_x = math.sqrt(var_x)
    sd_y = math.sqrt(var_y)

    co_xy = cov / (sd_x * sd_y)

    return co_xy

In [18]:
def selection_accurecy_svm(selected_features):
    x = data[selected_features];
    y = data['class'];

    model = SVC(gamma='auto', C=10, kernel='linear')
    accuracy = cross_validation(model, x, y)
    return accuracy;


In [19]:
def selection_accurecy_dt(selected_features):

    x = data[selected_features];
    y = data['class'];

    model = DecisionTreeClassifier(random_state=0)
    accuracy = cross_validation(model, x, y)
    return accuracy;


In [20]:
def selection_accurecy_KNN(selected_features):

  x = data[selected_features];
  y = data['class'];

  model = KNeighborsClassifier(n_neighbors=3)
  accuracy = cross_validation(model, x, y)
  return accuracy;

  return accuracy_score(y_test, predictions)


In [21]:
def selection_accurecy_nb(selected_features):

  x = data[selected_features];
  y = data['class'];

  model = GaussianNB()

  accuracy = cross_validation(model, x, y)
  return accuracy;

In [22]:
def selection_accurecy_xgb(selected_features):
    x = data[selected_features];
    y = data['class'];

    model = XGBClassifier()

    accuracy = cross_validation(model, x, y)
    return accuracy;

In [23]:
def cross_validation(model, _X, _y):
    _scoring = ['accuracy', 'precision', 'recall', 'test_score']
    results = cross_validate(estimator=model,
                             X=_X,
                             y=_y,
                             cv=10,
                             # scoring=_scoring,
                             return_train_score=True)

    return results['test_score'].mean() * 100

In [24]:
def ret_covariance(x,y):
    return covariance_list[x][y]


In [25]:
main_data = pd.read_csv('datasets/unsw_nb_15_cat_in_num_binaryclass_50k_sample.csv')
main_data.head()

,0,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,...,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm
0,0,3,3,0.028016,3926,56526,31,29,7,27,...,0.0,0.0,0.0,13,4,2,7,1,1,2
1,0,3,3,1.007838,1580,10168,31,29,3,5,...,1.0,0.0,0.0,2,1,3,4,1,1,1
2,0,3,3,0.037968,2766,27384,31,29,7,16,...,0.0,0.0,0.0,16,6,5,5,1,1,3
3,1,4,2,13.790198,12832,0,254,0,0,0,...,0.0,0.0,0.0,1,1,1,1,1,1,1
4,0,3,3,0.018681,1540,1644,31,29,4,4,...,0.0,0.0,0.0,6,12,12,9,6,1,6


In [26]:
main_data.columns

Index(['0', 'proto', 'state', 'dur', 'sbytes', 'dbytes', 'sttl', 'dttl',
       'sloss', 'dloss', 'service', 'sload', 'dload', 'spkts', 'dpkts', 'swin',
       'dwin', 'stcpb', 'dtcpb', 'smeansz', 'dmeansz', 'trans_depth',
       'res_bdy_len', 'sjit', 'djit', 'stime', 'ltime', 'sintpkt', 'dintpkt',
       'tcprtt', 'synack', 'ackdat', 'is_sm_ips_ports', 'ct_state_ttl',
       'ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd', 'ct_srv_src',
       'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ltm', 'ct_src_dport_ltm',
       'ct_dst_sport_ltm', 'ct_dst_src_ltm'],
      dtype='object')

In [27]:
main_data.shape

(49999, 44)

In [28]:
rows_with_nan = main_data[main_data.isnull().any(axis=1)]
print(rows_with_nan)
main_data = main_data.dropna()

non_numerical_columns = main_data.select_dtypes(exclude=['number'])
non_numerical_columns

Empty DataFrame
Columns: [0, proto, state, dur, sbytes, dbytes, sttl, dttl, sloss, dloss, service, sload, dload, spkts, dpkts, swin, dwin, stcpb, dtcpb, smeansz, dmeansz, trans_depth, res_bdy_len, sjit, djit, stime, ltime, sintpkt, dintpkt, tcprtt, synack, ackdat, is_sm_ips_ports, ct_state_ttl, ct_flw_http_mthd, is_ftp_login, ct_ftp_cmd, ct_srv_src, ct_srv_dst, ct_dst_ltm, ct_src_ltm, ct_src_dport_ltm, ct_dst_sport_ltm, ct_dst_src_ltm]
Index: []

[0 rows x 44 columns]


""
0
1
2
3
4
...
49994
49995
49996
49997


In [29]:
class_column = '0' #might need to edit this depending on dataset

In [30]:
feature_list = main_data
feature_list =  feature_list.drop(class_column, axis=1)
feature_list = feature_list.columns

x = main_data[feature_list]
y = main_data[class_column]

feature_list = main_data.columns

In [31]:
caim = CAIMD()

x_disc = caim.fit_transform(x, y)

Categorical [14, 20, 31, 34]
# 0  GLOBAL CAIM  19698.96860800065
# 1  GLOBAL CAIM  19346.623547682168
# 2  GLOBAL CAIM  20350.765250883967
# 3  GLOBAL CAIM  22746.154343306458
# 4  GLOBAL CAIM  20326.88745604856
# 5  GLOBAL CAIM  24145.30555174917
# 6  GLOBAL CAIM  19604.953512205066
# 7  GLOBAL CAIM  19375.16483686022
# 8  GLOBAL CAIM  19415.419523265515
# 9  GLOBAL CAIM  19106.946386482137
# 10  GLOBAL CAIM  20425.44994057103
# 11  GLOBAL CAIM  20321.53085044143
# 12  GLOBAL CAIM  19462.903064847582
# 13  GLOBAL CAIM  20322.806889333457
# 15  GLOBAL CAIM  19352.849569774182
# 16  GLOBAL CAIM  19353.217168875715
# 17  GLOBAL CAIM  19352.941897959514
# 18  GLOBAL CAIM  19855.90937076841
# 19  GLOBAL CAIM  20433.670090528576
# 21  GLOBAL CAIM  19095.965062512903
# 22  GLOBAL CAIM  19387.931610106087
# 23  GLOBAL CAIM  19455.566162333733
# 24  GLOBAL CAIM  19293.98303167044
# 25  GLOBAL CAIM  19293.98303167044
# 26  GLOBAL CAIM  19526.08693974442
# 27  GLOBAL CAIM  20132.836018993148
# 2

In [32]:
data = x_disc
X = x_disc.iloc[:, :-1]
data['class'] = y
print(data.shape)

(49999, 44)


In [33]:
feature_list = data.columns

In [34]:
xk_mim = [32, 5, 3, 19, 13, 4, 27, 11, 2, 10, 41, 42, 40, 37, 36, 8, 26, 23, 38, 39, 22, 14, 16, 17, 15, 25, 24, 7, 9, 1, 0, 6, 30, 28, 29, 21, 20, 35, 34, 18, 31, 12, 33]

In [35]:
xk_mrmr = [32, 41, 3, 0, 5, 1, 25, 6, 19, 33, 35, 28, 10, 21, 31, 27, 30, 12, 8, 2, 29, 34, 13, 18, 39, 24, 20, 4, 26, 11, 38, 22, 36, 40, 37, 7, 23, 42, 14, 9, 16, 15, 17]

In [36]:
xk_dimrmr = [32, 9, 3, 0, 5, 25, 6, 19, 1, 33, 41, 28, 31, 10, 35, 21, 27, 30, 12, 2, 29, 34, 13, 4, 18, 8, 11, 24, 20, 39, 42, 26, 40, 36, 37, 38, 22, 23, 7, 14, 16, 15, 17]

In [37]:
df = pd.read_csv('results/unsw_50ksample_comparison.csv')

# Print the DataFrame
print(df)

# Retrieve data from the DataFrame
mim_vals = df['MIM'].tolist()
mrmr_vals = df['MRMR'].tolist()
dimrmr_vals = df['DIMRMR'].tolist()

         MIM      MRMR    DIMRMR        DI   C_ratio
0   0.042581  0.025264  0.014490  0.536720  0.000475
1   0.046015  0.006431  0.001278  0.342115 -0.013528
2   0.198944 -0.004363 -0.000392  0.087833 -0.067572
3   0.278115  0.042785  0.014991  0.203938 -0.143220
4   0.203674 -0.012937 -0.002438  0.081386 -0.070195
5   0.319889  0.084611  0.015764  0.117418 -0.145450
6   0.038481  0.012474  0.002696  0.415933  0.001901
7   0.064071 -0.076574 -0.015972  0.164931 -0.023621
8   0.099020  0.002772 -0.002754  0.155103 -0.033152
9   0.056452 -0.125288  0.013778  0.852351 -0.032513
10  0.198175  0.011823  0.000871  0.114348 -0.071454
11  0.203633 -0.033934 -0.002931  0.072643 -0.073333
12  0.000104 -0.002223 -0.000519  0.242897  0.000935
13  0.203840 -0.008564 -0.000676  0.085475 -0.064141
14  0.075141 -0.111763 -0.018345  0.151206 -0.029184
15  0.074582 -0.162245 -0.023225  0.143149 -0.031640
16  0.074618 -0.143449 -0.021065  0.146845 -0.030561
17  0.074604 -0.179924 -0.025122  0.139628 -0.

In [38]:
!python --version

Python 3.10.12


In [40]:
# all features
parameters = {'n_estimators': [200, 500], 'max_features': ['sqrt', 'log2'], 'max_depth': [4, 5, 6, 7, 8], 'criterion':['gini', 'entropy', 'log_loss']}
rfc=RandomForestClassifier(random_state=42)
clf = GridSearchCV(rfc, parameters, error_score='raise')
features = xk_dimrmr
print(features)
x = data.iloc[:, features]
y = data['class']
# print(x)
# print(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1, stratify=y)
clf.fit(x_train, y_train)
sorted(clf.cv_results_)
print(clf.best_params_)
predictions = clf.predict(x_test)
print("Random Forest Accuracy")
print(accuracy_score(y_test, predictions))

[32, 9, 3, 0, 5, 25, 6, 19, 1, 33, 41, 28, 31, 10, 35, 21, 27, 30, 12, 2, 29, 34, 13, 4, 18, 8, 11, 24, 20, 39, 42, 26, 40, 36, 37, 38, 22, 23, 7, 14, 16, 15, 17]
{'criterion': 'entropy', 'max_depth': 6, 'max_features': 'log2', 'n_estimators': 500}
Random Forest Accuracy
0.9889


In [ ]:
# mrmr top five features
parameters = {'n_estimators': [200, 500], 'max_features': ['sqrt', 'log2'], 'max_depth': [4, 5, 6, 7, 8], 'criterion':['gini', 'entropy', 'log_loss']}
rfc=RandomForestClassifier(random_state=42)
clf = GridSearchCV(rfc, parameters, error_score='raise')
features = xk_mrmr[:5]
print(features)
x = data.iloc[:, features]
y = data['class']
print(x)
# print(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1, stratify=y)
clf.fit(x_train, y_train)
sorted(clf.cv_results_)
print(clf.best_params_)
predictions = clf.predict(x_test)
print("Random Forest Accuracy")
print(accuracy_score(y_test, predictions))

[32, 41, 3, 0, 5]
       ct_state_ttl  ct_dst_sport_ltm  sbytes  proto  sttl
0               0.0               0.0     1.0    0.0   0.0
1               0.0               0.0     1.0    0.0   0.0
2               0.0               0.0     1.0    0.0   0.0
3               1.0               0.0     1.0    0.0   1.0
4               0.0               0.0     1.0    0.0   0.0
...             ...               ...     ...    ...   ...
49994           0.0               1.0     1.0    0.0   0.0
49995           0.0               1.0     1.0    0.0   0.0
49996           0.0               0.0     1.0    0.0   0.0
49997           0.0               0.0     1.0    0.0   0.0
49998           0.0               0.0     1.0    0.0   0.0

[49999 rows x 5 columns]
{'criterion': 'gini', 'max_depth': 5, 'max_features': 'sqrt', 'n_estimators': 200}
Random Forest Accuracy
0.9869


In [41]:
# dimrmr top five features
parameters = {'n_estimators': [200, 500], 'max_features': ['sqrt', 'log2'], 'max_depth': [4, 5, 6, 7, 8], 'criterion':['gini', 'entropy', 'log_loss']}
rfc=RandomForestClassifier(random_state=42)
clf = GridSearchCV(rfc, parameters, error_score='raise')
features = xk_dimrmr[:5]
print(features)
x = data.iloc[:, features]
y = data['class']
print(x)
# print(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1, stratify=y)
clf.fit(x_train, y_train)
sorted(clf.cv_results_)
print(clf.best_params_)
predictions = clf.predict(x_test)
print("Random Forest Accuracy")
print(accuracy_score(y_test, predictions))

[32, 9, 3, 0, 5]
       ct_state_ttl  service  sbytes  proto  sttl
0               0.0      1.0     1.0    0.0   0.0
1               0.0      1.0     1.0    0.0   0.0
2               0.0      1.0     1.0    0.0   0.0
3               1.0      1.0     1.0    0.0   1.0
4               0.0      1.0     1.0    0.0   0.0
...             ...      ...     ...    ...   ...
49994           0.0      1.0     1.0    0.0   0.0
49995           0.0      1.0     1.0    0.0   0.0
49996           0.0      1.0     1.0    0.0   0.0
49997           0.0      1.0     1.0    0.0   0.0
49998           0.0      1.0     1.0    0.0   0.0

[49999 rows x 5 columns]
{'criterion': 'gini', 'max_depth': 5, 'max_features': 'sqrt', 'n_estimators': 200}
Random Forest Accuracy
0.9874


In [42]:
# dimrmr top 2 features
parameters = {'n_estimators': [200, 500], 'max_features': ['sqrt', 'log2'], 'max_depth': [4, 5, 6, 7, 8], 'criterion':['gini', 'entropy', 'log_loss']}
rfc=RandomForestClassifier(random_state=42)
clf = GridSearchCV(rfc, parameters, error_score='raise')
features = xk_dimrmr[:2]
print(features)
x = data.iloc[:, features]
y = data['class']
print(x)
# print(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1, stratify=y)
clf.fit(x_train, y_train)
sorted(clf.cv_results_)
print(clf.best_params_)
predictions = clf.predict(x_test)
print("Random Forest Accuracy")
print(accuracy_score(y_test, predictions))

[32, 9]
       ct_state_ttl  service
0               0.0      1.0
1               0.0      1.0
2               0.0      1.0
3               1.0      1.0
4               0.0      1.0
...             ...      ...
49994           0.0      1.0
49995           0.0      1.0
49996           0.0      1.0
49997           0.0      1.0
49998           0.0      1.0

[49999 rows x 2 columns]
{'criterion': 'gini', 'max_depth': 4, 'max_features': 'sqrt', 'n_estimators': 200}
Random Forest Accuracy
0.9828


In [44]:
# improvized  features
parameters = {'n_estimators': [200, 500], 'max_features': ['sqrt', 'log2'], 'max_depth': [4, 5, 6, 7, 8], 'criterion':['gini', 'entropy', 'log_loss']}
rfc=RandomForestClassifier(random_state=42)
clf = GridSearchCV(rfc, parameters, error_score='raise')
features = [17]
print(features)
x = data.iloc[:, features]
y = data['class']
print(x)
# print(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1, stratify=y)
clf.fit(x_train, y_train)
sorted(clf.cv_results_)
print(clf.best_params_)
predictions = clf.predict(x_test)
print("Random Forest Accuracy")
print(accuracy_score(y_test, predictions))

[17]
       dtcpb
0        1.0
1        1.0
2        1.0
3        0.0
4        1.0
...      ...
49994    1.0
49995    1.0
49996    1.0
49997    0.0
49998    1.0

[49999 rows x 1 columns]
{'criterion': 'gini', 'max_depth': 4, 'max_features': 'sqrt', 'n_estimators': 200}
Random Forest Accuracy
0.8735


In [ ]:
# improvized five features
parameters = {'n_estimators': [200, 500], 'max_features': ['sqrt', 'log2'], 'max_depth': [4, 5, 6, 7, 8], 'criterion':['gini', 'entropy', 'log_loss']}
rfc=RandomForestClassifier(random_state=42)
clf = GridSearchCV(rfc, parameters, error_score='raise')
features = [32, 9, 3, 30]
print(features)
x = data.iloc[:, features]
y = data['class']
print(x)
# print(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1, stratify=y)
clf.fit(x_train, y_train)
sorted(clf.cv_results_)
print(clf.best_params_)
predictions = clf.predict(x_test)
print("Random Forest Accuracy")
print(accuracy_score(y_test, predictions))

[32, 9, 3, 30]
       ct_state_ttl  service  sbytes  ackdat
0               0.0      1.0     1.0     0.0
1               0.0      1.0     1.0     0.0
2               0.0      1.0     1.0     0.0
3               1.0      1.0     1.0     0.0
4               0.0      1.0     1.0     0.0
...             ...      ...     ...     ...
49994           0.0      1.0     1.0     0.0
49995           0.0      1.0     1.0     0.0
49996           0.0      1.0     1.0     0.0
49997           0.0      1.0     1.0     0.0
49998           0.0      1.0     1.0     0.0

[49999 rows x 4 columns]
{'criterion': 'gini', 'max_depth': 4, 'max_features': 'sqrt', 'n_estimators': 200}
Random Forest Accuracy
0.9867
